In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab07.ipynb")

# Lab 7: Great British Bake Off (A/B Test)

Welcome to Data 8 Lab 7! This week's lab will focus on A/B Testing using data from the ever-popular British television show, [*The Great British Bakeoff*](https://en.wikipedia.org/wiki/The_Great_British_Bake_Off).

**Lab Queue**: You can find the Lab Queue at [lab.data8.org](https://lab.data8.org/). Whenever you feel stuck or need some further clarification, add yourself to the queue to get help from a GSI or academic intern! Please list your name, location, and purpose on your ticket!

**Grading**: Here are the policies for getting full credit:

1. To receive credit for this lab notebook (8% final grade), you must submit it by **Friday March 18th at 5pm**.

2. To receive credit for the lab attendance (2% final grade), you must attend at least 5 out of the 14 labs.

**Submission**: Once you're finished, scroll up to the top of the page, and run every cell in the notebook.  Select "Save" in the File menu (please don’t hit Save As), then rerun the last cell of the notebook.  The result will contain a zip file that you can submit to Gradescope. If you are in lab, you can submit a ticket on lab.data8.org to ask a staff member to check you off. 

First, set up the notebook by running the cell below.

In [2]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', (FutureWarning, np.VisibleDeprecationWarning))

import d8error

## 1. A/B Testing

A/B testing is a form of hypothesis testing that allows you to make comparisons between two distributions. We may also refer to an A/B test as a permutation test.

You'll almost never be explicitly asked to perform an A/B test. Make sure you can identify situations where the test is appropriate and know how to correctly implement each step. Oftentimes, we use an A/B test to determine whether or not two samples came from the same underlying distribution.

**Question 1.1.** The following statements are the steps of an A/B hypothesis test presented in a *random order*:

1. Choose a test statistic (typically the difference in means between two categories)

2. Shuffle the labels of the original sample, find your simulated test statistic, and repeat many times

3. Find the value of the observed test statistic

4. Calculate the p-value based off your observed and simulated test statistics

5. Define a null and alternate model

6. Use the p-value and p-value cutoff to draw a conclusion about the null hypothesis

Assign `ab_test_order` to an array of integers that contains the correct order of an A/B test, where the first item of the array is the first step of an A/B test and the last item of the array is the last step of an A/B test.

<!--
BEGIN QUESTION
name: q1_1
-->

In [3]:
ab_test_order = make_array(5, 1, 3, 2, 4, 6)

In [4]:
grader.check("q1_1")

q1_1 results: All test cases passed!

**Question 1.2.** If the null hypothesis of an A/B test is correct, should the order of labels affect the differences in means between each group? Why do we shuffle labels in an A/B test? If you are in a lab section, confirm your answer with a neighbor or staff member before moving on. 

<!--
BEGIN QUESTION
name: q1_2
-->

_If the null hypothesis of an A/B test is correct, the order of labels won't affect the differences in means. We shuffle the labels in order to perform the permutation tests to observe to see if there is an effect of assinging the labels to different values. If random sampling does not effect it, then it should not make a difference for the A/B/ test._

## Check-in

When you are ready to answer this question, please call your GSI or AI over to check in and answer: When does an A/B test show that the treatment causes a change in outcome?

## 2. The Great British Bake Off

>"The Great British Bake Off (often abbreviated to Bake Off or GBBO) is a British television baking competition, produced by Love Productions, in which a group of amateur bakers compete against each other in a series of rounds, attempting to impress a group of judges with their baking skills" [Wikipedia](https://en.wikipedia.org/wiki/The_Great_British_Bake_Off)

For every week of the competition, the judges assign one contestant the title "Star Baker". Ultimately, one winner is crowned every season. Using this information, we would like to investigate how winning Star Baker awards affects the odds of winning a season of the show.

**Question 2.1.** We want to know whether winning more Star Baker awards *causes* a change in likelihood of winning the season.  Why is it not sufficient to compare star baker rates for winners and losers?

<!--
BEGIN QUESTION
name: q2_1
-->

_It is not sufficient to compare star baker rates for winners and losers because we also need to control for confounding variables before we can make any causal statements. You can compare those two, but cannot assume a causal statement or stance without random sampling, which controls for other confounding variables (this is the benefit of the permutation test)!_

### Running an Experiment

We are going to run the following hypothesis test to determine the association between winning and number of Star Baker awards. The population we are examining is every contestant from seasons 2 through 11 of GBBO. We are going to use the following null and alternative hypotheses:

**Null hypothesis:** The distribution of Star Baker awards between contestants who won their season and contestants who did not win their season is the same.

**Alternative hypothesis:** Contestants who win their season of the show will win more Star Baker awards on average.

Our alternative hypothesis is related to our suspicion that contestants who win more Star Baker awards are more skilled, so they are more likely to win the season.

**Question 2.2.** Should we use an A/B test to test these hypotheses? If yes, what is our "A" group and what is our "B" group?

<!--
BEGIN QUESTION
name: q2_2
-->

_Yes, we should use an A/B test to test whether the two samples come from the same distribution. The A group would be the distribution of Star Baker awards between contestants who won their season and sample B would be the Star Baker awards between contestants who did not win their season._

Check your answers with your neighbors or a staff member before you move on to the next section.

The `bakers` table below describes the number of star baker awards each contest won and whether or not they won their season (`1` if they won, `0` if they did not win). The data was manually aggregated from Wikipedia for seasons 2-11 of the show. We randomized the order of rows as to not spoil the outcome of the show.

In [5]:
bakers = Table.read_table("star_bakers.csv")
bakers.show(3)

**Question 2.3.** Create a new table called `means` that contains the mean number of star baker awards for bakers who did not win (`won==0`) and bakers that did win (`won==1`). The table should have the column names `won` and `star baker awards mean`.
<!--
BEGIN QUESTION
name: q2_3
-->

In [6]:
means = bakers.group("won", np.average).relabeled("star baker awards average", "star baker awards mean")
means

won  | star baker awards mean
0    | 0.651786
1    | 1.5

In [7]:
grader.check("q2_3")

q2_3 results: All test cases passed!

**Question 2.4.** Visualize the distribution of Star Baker awards for winners and non-winners. You should use the bins we provided.

Hint: You will want to use the group argument of `tbl.hist`. In order to produce several overlayed histograms based on unique values in a given column, we can do something like `tbl.hist(..., group=<col_name>, bins=...)`!

<!--
BEGIN QUESTION
name: q2_4
-->

In [8]:
useful_bins = np.arange(0, 7)
bakers.hist("star baker awards",group = "won", bins = useful_bins)

**Question 2.5.** We want to figure out if there is a difference between the distribution of Star Baker awards between winners and non winners. 

What should the test statistic be? Which values of this test statistic support the null, and which values support the alternative?

If you are in lab, confirm your answer with a neighbor or staff member before moving on.

Hint: You should think about what measures we use to describe a distribution. 

<!--
BEGIN QUESTION
name: q2_5
-->

_The test statistic should be the difference in average star baker awards for those who won their season and those who did not win their seasons. This is basically Group A average - Group B average. Values of this test statistic that support the null are equal to zero or below. Values of this test statistic that support the alternative would be values that are positive._

**Question 2.6.** Set `observed_difference` to the observed test statistic using the `means` table. 

<!--
BEGIN QUESTION
name: q2_6
-->

In [9]:
observed_difference = means.column("star baker awards mean").item(1) - means.column("star baker awards mean").item(0)
observed_difference

0.8482142857142857

In [10]:
grader.check("q2_6")

q2_6 results: All test cases passed!

**Question 2.7.** Given a table like `bakers`, a label column `label_col`, and a values column `val_col`, write a function that calculates the appropriate test statistic.

*Hint:* Make sure that you are taking the directionality of our alternative hypothesis into account.

<!--
BEGIN QUESTION
name: q2_7
-->

In [11]:
def find_test_stat(tbl, label_col, val_col):
    means_table = tbl.group(label_col, np.average)
    means = means_table.column(1)
    return means.item(1) - means.item(0)

find_test_stat(bakers, "won", "star baker awards")

0.8482142857142857

In [12]:
grader.check("q2_7")

q2_7 results: All test cases passed!

When we run a simulation for A/B testing, we resample by **shuffling the labels** of the original sample. If the null hypothesis is true and the star baker award distributions are the same, we expect that the difference in mean star baker awards will be not change when `"won"` labels are changed.

**Question 2.8.** Write a function `simulate_and_test_statistic` to compute one trial of our A/B test. Your function should run a simulation and return a test statistic.

<!--
BEGIN QUESTION
name: q2_8
-->

In [16]:
def simulate_and_test_statistic(tbl, labels_col, values_col):
    shuffled_labels = tbl.sample(with_replacement = False).column(labels_col)
    shuffled_table = tbl.select(values_col).with_column('Shuffled Label', shuffled_labels)
    return find_test_stat(shuffled_table, 'Shuffled Label', values_col)

simulate_and_test_statistic(bakers, "won", "star baker awards")

0.3035714285714286

In [17]:
grader.check("q2_8")

q2_8 results: All test cases passed!

**Question 2.9.** Simulate 5000 trials of our A/B test and store the test statistics in an array called `differences`.

<!--
BEGIN QUESTION
name: q2_9
-->

In [20]:
# This cell might take a couple seconds to run
differences = make_array()

repetitions = 5000

for i in np.arange(repetitions):
    new_diff = simulate_and_test_statistic(bakers, "won", "star baker awards")
    differences = np.append(differences, new_diff)
                                                 
differences

array([-0.35      , -0.13214286,  0.63035714, ...,  0.95714286,
        0.73928571,  0.19464286])

In [21]:
grader.check("q2_9")

q2_9 results: All test cases passed!

Run the cell below to view a histogram of your simulated test statistics plotted with your observed test statistic.

In [22]:
Table().with_column('Difference Between Group Means', differences).hist(bins=20)
plots.scatter(observed_difference, 0, color='red', s=30, zorder=2)
plots.ylim(-0.1, 1.35);

**Question 2.10.** Find the p-value for your test and assign it to `empirical_p`.

<!--
BEGIN QUESTION
name: q2_10
-->

In [23]:
empirical_p = np.count_nonzero(differences >= observed_difference) / repetitions
empirical_p

0.0136

In [24]:
grader.check("q2_10")

q2_10 results: All test cases passed!

**Question 2.11.** Using a 5% P-value cutoff, draw a conclusion about the null and alternative hypotheses. Describe your findings using simple, non-technical language. What does your analysis tell you about the association between star baker awards and winning? What can you claim about causation from your statistical analysis? Call your GSI or AI over and confirm your answer. 

<!--
BEGIN QUESTION
name: q2_10
-->

Using a 5% p-value cutoff, we can reject the null hypothesis because the p value of this test is 1.36%. The findings are that contestants who win their season of the show will win more Star Baker awards on average rather than being the same distribution as contestants who do not win their season of the show. The association between star baker awards and winning is positive. I can claim a causal statement that winning more Star Baker awards causes people to be more likely to win the season of the show (because we did random sampling and thus can confirm a causal statement). 

## 3. Submission

Minnie wanted to say congrats on finishing Lab 7!! You can now be like Minnie and enjoy your spring break!

<img src="minnie_car.jpg" alt="drawing" width="300"/>

<img src="minnie_beach.jpg" alt="drawing" width="300"/>

Be sure to...

* **run all the tests** (or run the `grader.check_all` cell),
* **Save and Checkpoint** from the File menu,
* **run the last cell to generate the zip file and submit your work to Lab 07 Autograder on [Gradescope](https://www.gradescope.com/courses/288788)**,
* and if you are in lab, **ask one of the staff members to check you off.**

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)